In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/My Drive/Colab Notebooks"
os.chdir(cwd)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random

torch.manual_seed(1)
random.seed(1)

In [11]:
torch.manual_seed(1)
random.seed(1)

#get inputs of any shape, gets outputs of a certain vector size, simple categorical classification 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size, hidden_size)      
        self.hidden2out=nn.Linear(hidden_size,output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        self.softmax = nn.LogSoftmax(dim=1) #new
    def forward(self, x): #need to iterate over each item in the input tensor
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        outs = self.out(lstm_out)
        return outs
    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))
        
n_input=4
n_hidden=16
n_output=15
rnn = RNN(n_input, n_hidden, n_output)
loss_func = nn.MSELoss()
LR=0.005
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters
rand_tensor = 0.2*torch.rand((3, 4))
rnn.hidden = rnn.init_hidden()
rnn.zero_grad()
for i in range(len(rand_tensor)): #feed the network sequentially with the input tensors
    cur_tensor=rand_tensor[i].view([1,1,n_input])
    output = rnn(cur_tensor)
print("input shape", rand_tensor.shape)
print("output shape", output.shape)
print(output)


input shape torch.Size([3, 4])
output shape torch.Size([1, 1, 15])
tensor([[[-0.0327, -0.1762, -0.2155,  0.1499, -0.2015,  0.2665, -0.0605,
          -0.1984, -0.1878,  0.2387, -0.2962,  0.0704,  0.1602,  0.1239,
          -0.2588]]], grad_fn=<AddBackward0>)


In [16]:
#here the size of the output is the same as the size of the input
#the depth of the output depends on the number of possible outcome categories (e.g. different phonemes)

torch.manual_seed(1)
random.seed(1)

class RNN(nn.Module):

    def __init__(self, input_size, hidden_size, output_depth,batch_size=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.batch_size=batch_size
        self.lstm = nn.LSTM(input_size, hidden_size)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2out = nn.Linear(hidden_size, output_depth)
        self.hidden = self.init_hidden()

    def forward(self, feature_list): #emeds are the list of features for each word in the sentece
        #sent_size=len(embeds)
        lstm_out, _ = self.lstm( feature_list.view(len( feature_list), 1, -1))
        tag_space = self.hidden2out(lstm_out.view(len( feature_list), -1))
        #print(tag_space.view([1,1,1]))
        tag_scores = torch.sigmoid(tag_space)
        #return tag_scores
        return tag_space
       
    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))  

n_input=4
n_hidden =16

rnn = RNN(n_input, n_hidden, 5)
line_tensor=torch.rand((10, 4))
output = rnn(line_tensor)

print("input shape", line_tensor.shape)
print("output shape", output.shape)
print(output)

rnn = RNN(n_input, n_hidden, 1)
line_tensor=torch.rand((10, 4))
output = rnn(line_tensor)

print("input shape", line_tensor.shape)
print("output shape", output.shape)
print(output)


input shape torch.Size([10, 4])
output shape torch.Size([10, 5])
tensor([[-0.1749,  0.1472,  0.1203,  0.1388, -0.0397],
        [-0.1918,  0.1713,  0.1225,  0.1205, -0.0532],
        [-0.1963,  0.1876,  0.1238,  0.1076, -0.0607],
        [-0.2091,  0.1810,  0.1401,  0.1114, -0.0629],
        [-0.2091,  0.1674,  0.1404,  0.1202, -0.0650],
        [-0.2226,  0.2021,  0.1504,  0.0986, -0.0750],
        [-0.2159,  0.1999,  0.1501,  0.1009, -0.0739],
        [-0.2073,  0.1808,  0.1418,  0.1121, -0.0662],
        [-0.2214,  0.1858,  0.1528,  0.1088, -0.0685],
        [-0.2135,  0.1928,  0.1424,  0.1020, -0.0730]],
       grad_fn=<AddmmBackward>)
input shape torch.Size([10, 4])
output shape torch.Size([10, 1])
tensor([[-0.1060],
        [-0.1050],
        [-0.0985],
        [-0.0989],
        [-0.0933],
        [-0.0899],
        [-0.0833],
        [-0.0829],
        [-0.0923],
        [-0.0943]], grad_fn=<AddmmBackward>)


In [13]:
#now we want to just do regression to get one value from a tensor
torch.manual_seed(1)
random.seed(1)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size=1):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.batch_size = batch_size

        self.lstm = nn.LSTM(input_size, hidden_size)      
        #self.lstm.cuda(cuda0)  
        self.hidden2out=nn.Linear(hidden_size,output_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        self.softmax = nn.LogSoftmax(dim=1) #new
        
    def forward(self, x):
        #self.hidden
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        outs = self.out(lstm_out)
        #outs = self.softmax(outs)
        return outs#, h_state


    def init_hidden(self):
        return (torch.zeros(1, self.batch_size, self.hidden_size),
                torch.zeros(1, self.batch_size, self.hidden_size))

n_input=4
n_hidden = 16
rnn = RNN(n_input, n_hidden, 1)
rnn.hidden = rnn.init_hidden()
rnn.zero_grad()
line_tensor=torch.rand((10, 4))

for i in range(line_tensor.size()[0]):
  cur_tensor=line_tensor[i]#.view([(1, 1, 3)])
  cur_tensor=cur_tensor.view([1,1,n_input])
  output = rnn(cur_tensor)
print(output)

tensor([[[0.1352]]], grad_fn=<AddBackward0>)
